# Dt/dtaumix study

In [ ]:
import sys
githubPath = r"K:/Dropbox (GaTech)/DOE/BatchPaSR"
sys.path.insert(0, githubPath)
import BatchPaSR as bp; 
import CanteraTools as ct_tools; 
import cantera as ct;
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
phi_global = 0.635
phi_main = 0.3719
tau_sec = 0.28
tau_global = 20
tau_mix = [0.005, 0.01, 0.05, 0.1, 0.3]
P = 25 * ct.one_atm
miliseconds = 0.001

In [ ]:
#making figures
fig1, (Tvst, COvt, NOvt) = plt.subplots(nrows=3,ncols=1)
plt.xlabel('Time (seconds)')
Tvst.set_title("Average Tempature over Time for Tau_mix = 0.005")
COvt.set_title('Average CO Concentration over time for Tau_mix = 0.005 (uncorrected ppm)')
NOvt.set_title('Average NO Concentration over time for Tau_mix = 0.005 (uncorrected ppm)')

fig2, (Tvstq, COvtq, NOvtq) = plt.subplots(nrows=3,ncols=1)
plt.xlabel('Time (seconds)')
Tvstq.set_title("Average Tempature over Time for Tau_mix = 0.05")
COvtq.set_title('Average CO Concentration over time for Tau_mix = 0.05 (uncorrected ppm)')
NOvtq.set_title('Average NO Concentration over time for Tau_mix = 0.05 (uncorrected ppm)')

fig3, (Tvstw, COvtw, NOvtw) = plt.subplots(nrows=3,ncols=1)
plt.xlabel('Time (seconds)')
Tvstw.set_title("Average Tempature over Time for Tau_mix = 0.01")
COvtw.set_title('Average CO Concentration over time for Tau_mix = 0.01 (uncorrected ppm)')
NOvtw.set_title('Average NO Concentration over time for Tau_mix = 0.01 (uncorrected ppm)')

fig4, (Tvste, COvte, NOvte) = plt.subplots(nrows=3,ncols=1)
plt.xlabel('Time (seconds)')
Tvste.set_title("Average Tempature over Time for Tau_mix = 0.1")
COvte.set_title('Average CO Concentration over time for Tau_mix = 0.1 (uncorrected ppm)')
NOvte.set_title('Average NO Concentration over time for Tau_mix = 0.1 (uncorrected ppm)')

fig5, (Tvstr, COvtr, NOvtr) = plt.subplots(nrows=3,ncols=1)
plt.xlabel('Time (seconds)')
Tvstr.set_title("Average Tempature over Time for Tau_mix = 0.3")
COvtr.set_title('Average CO Concentration over time for Tau_mix = 0.3 (uncorrected ppm)')
NOvtr.set_title('Average NO Concentration over time for Tau_mix = 0.3 (uncorrected ppm)')

In [ ]:
for y in range(0,8):

    for x in range(0,5):
    #note add somewhere on graph that tau_mix is in miliseconds
    #add somewhere to include # of particles
        [mfm, mam, mfs, mas] = solvePhi_airSplit(phi_global, phi_main, 100, 1)
        [vit_reactor, main_burner_DF] = runMainBurner(phi_main, (tau_global-tau_sec)*milliseconds)
        vit_particle = Particle.from_gas(vit_reactor.thermo, particle_mass = mfm+mam)

        ms = miliseconds+y*1e-3
        dt = miliseconds * ms # in milliseconds
        secondary_gas = ct.Solution('gri30.xml')
        secondary_gas.TPX = 300, 25*ct.one_atm, {'CH4':1}
        secondary_part = Particle.from_gas(secondary_gas, particle_mass=mfs+mas)

        bp = LiME(particle_list=[vit_particle, secondary_part], dt=dt, N_MAX=1000)
        states = ct.SolutionArray(bp.mean_gas, extra=['t'])
        enthalpy = []
        t = np.arange(0, .4*milliseconds, bp.dt)

        #bp.prep_entrainment(added_gas = vit_reactor.thermo, total_mass_added = (mam+mfm), tau_ent = totalTime, numParticles=20, method='constant')
        for i in range(0,t.size):
            #bp.entrain(t[i])
            bp.react()
            bp.mix(tau_mix=tau_mix[x]*milliseconds)
            states.append(bp.mean_gas.state, t=t[i])
            enthalpy.append(bp.mean_gas.enthalpy_mass)
        if x == 0:
            Tvst.plot(states.t, states.T, label = 'dt = %s' %(ms))
            COvt.plot(states.t, states('CO').X*1e6, label = 'dt = %s' %(ms))
            NOvt.plot(states.t, states('NO').X*1e6, label = 'dt = %s' %(ms))
            #NOvCO.plot(states('CO').X*1e6, states('NO').X*1e6, label = 'Tau_mix = %s' %(tau_mix[x]) )
        if x == 1:
            Tvstq.plot(states.t, states.T, label = 'dt = %s' %(ms))
            COvtq.plot(states.t, states('CO').X*1e6, label = 'dt = %s' %(ms))
            NOvtq.plot(states.t, states('NO').X*1e6, label = 'dt = %s' %(ms))
        if x == 2:
            Tvstw.plot(states.t, states.T, label = 'dt = %s' %(ms))
            COvtw.plot(states.t, states('CO').X*1e6, label = 'dt = %s' %(ms))
            NOvtw.plot(states.t, states('NO').X*1e6, label = 'dt = %s' %(ms))
        if x == 3:
            Tvste.plot(states.t, states.T, label = 'dt = %s' %(ms))
            COvte.plot(states.t, states('CO').X*1e6, label = 'dt = %s' %(ms))
            NOvte.plot(states.t, states('NO').X*1e6, label = 'dt = %s' %(ms))
        if x == 4:
            Tvstr.plot(states.t, states.T, label = 'dt = %s' %(ms))
            COvtr.plot(states.t, states('CO').X*1e6, label = 'dt = %s' %(ms))
            NOvtr.plot(states.t, states('NO').X*1e6, label = 'dt = %s' %(ms))

        
            

    
for ax in [COvt, COvte, COvtq, COvtr, COvtw, NOvt, NOvte, NOvtq, NOvtr, NOvtw, Tvst, Tvste, Tvstq, Tvstr, Tvstw]:
    ax.grid(True)
    ax.label_outer()
    ax.legend()

In [ ]:
fig1.savefig('Taum1')
fig2.savefig('Taum2')
fig3.savefig('Taum3')
fig4.savefig('Taum4')
fig5.savefig('Taum5')

In [ ]:
plt.show()